In [2]:
import numpy as np
import pandas as pd
import math as m
from datetime import datetime
#import onetimepad

In [3]:
# ----- Constantes Globais ------

#chave da criptografia
key = list(pd.read_csv('key.csv'))[0]

In [3]:
# ---------- Funcoes ------------

def cols2datetime(df):
    datas = []
    for x in df.loc[:,'Ano':'Minuto'].values :
        datas.append(datetime(*x))
    return datas


def otp(sr):
    return sr.apply(onetimepad.encrypt, key=key)   

def sort_and_replace(sr):
    
    sr.sort_values(inplace=True)    
    sr = sr.reset_index()
    sr.loc[:,'Placa'] = sr.groupby('Placa').ngroup()+1
    sr.set_index('index', inplace=True)

    return sr
    
    
def encriptar_placas(sr):
    return sort_and_replace(otp(sr))
    
def str2num(df, COLUMN):
    
    for column in COLUMN:
        df.loc[:, column] = pd.to_numeric(df.loc[:, column], errors='coerce')
    
    return df
    
def processar_encriptar_um_dia(lote, mes, dia):

    #Carrega um arquivo CSV num DataFrame
    df_dia = pd.read_csv("Trafego_CSV/"+lote+"/"+mes+"c/"+dia+".csv")
    
    #Deleta a coluna 'Velocidade Media'
    df_dia = df_dia.iloc[:,:-1]    
    
    #Insere a coluna unificada de data e hora no DataFrame
    datahora = cols2datetime(df_dia)
    df_dia['Data e Hora'] = datahora
    df_dia = df_dia.drop(columns=['Ano', 'Mes', 'Dia', 'Hora', 'Minuto', 'Segundo'])
    
    #Altera as colunas numericas de str para float
    df_dia = str2num(df_dia, ['Especie', 'Classe', 'Comprimento', 'Velocidade', 'Ocupacao'])
    
    #Cria coluna com final das Placa
    df_dia['Final de Placa'] = df_dia['Placa'].str[6]
    
    #Encripta as Placas
    placa_bool = df_dia['Placa']!='       '
    df_dia.loc[placa_bool, 'Placa'] = encriptar_placas(df_dia.loc[placa_bool, 'Placa'])
    
    #Reodena Colunas
    df_dia = df_dia[['Lote', \
                     'Data e Hora',\
                     'Local',\
                     'Faixa',\
                     'Entrefaixa',\
                     'Registro',\
                     'Tipo de Registro',\
                     'Placa',\
                     'Final de Placa', \
                     'Especie',\
                     'Classe',\
                     'Comprimento',\
                     'Velocidade',\
                     'Ocupacao']]
    
    #Renomeia a Coluna da Placas para Veiculo
    df_dia = df_dia.rename(columns={'Placa':'Veiculo'})
    
    #Escreve um novo CSV
    df_dia.to_csv("Trafego_CSV/"+lote+"/"+mes+"g/"+dia+".csv", index=False, float_format='%.0f')
    
    return df_dia

In [12]:
lote = 'L3'
mes ='01'
dia ='01'
df = pd.read_csv("Trafego_CSV/"+lote+"/"+mes+"c/"+dia+".csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
df[['Minuto', 'Faixa']]

,Minuto,Faixa
0,57,2
1,57,2
2,57,3
3,58,2
4,58,3
...,...,...
2862182,51,4
2862183,51,5
2862184,53,4
2862185,54,4


In [28]:
df['Teste1'] = 1*(df['Minuto'] < 58)
df['Teste2'] = 1*((df['Minuto'] < 58) & (df['Faixa']<4))
df[['Minuto', 'Faixa', 'Teste', 'Teste2']]

,Minuto,Faixa,Teste,Teste2
0,57,2,1,1
1,57,2,1,1
2,57,3,1,1
3,58,2,0,0
4,58,3,0,0
...,...,...,...,...
2862182,51,4,1,0
2862183,51,5,1,0
2862184,53,4,1,0
2862185,54,4,1,0


In [4]:
#----- Operacao -----

#parametros
lote = 'L3'
mes = '01'
'''
dias = ['01', '02', '03', '04', '05', \
        '06', '07', '08', '09', '10', \
        '11', '12', '13', '14', '15', \
        '16', '17', '18', '19', '20', \
        '21', '22', '23', '24', '25', \
        '26', '27', '28', '29', '30', '31']
'''
dias=['01']
for dia in dias:
    df = processar_encriptar_um_dia(lote, mes, dia)
    
#df.head(10)
#df.info()


/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
df_teste= pd.read_csv("Trafego_CSV/L3/01g/01.csv")
df_teste.head(10)

,Lote,Data e Hora,Local,Faixa,Entrefaixa,Registro,Tipo de Registro,Veiculo,Final de Placa,Especie,Classe,Comprimento,Velocidade,Ocupacao
0,L3,2017-12-31 23:57:00,5080,2,0,474948,0,541432,7,1.0,0.0,39.0,89.0,903.0
1,L3,2017-12-31 23:57:00,5080,2,0,474949,0,,,1.0,0.0,405.0,444.0,1249.0
2,L3,2017-12-31 23:57:00,5080,3,0,474950,0,,,1.0,0.0,118.0,97.0,1615.0
3,L3,2017-12-31 23:58:00,5080,2,0,474952,0,208568,0,1.0,0.0,39.0,61.0,1345.0
4,L3,2017-12-31 23:58:00,5080,3,0,474951,0,,,1.0,0.0,155.0,75.0,2848.0
5,L3,2017-12-31 23:58:00,5080,2,0,474953,0,160983,5,1.0,0.0,39.0,61.0,1412.0
6,L3,2017-12-31 23:59:00,5080,2,0,474954,0,546788,8,1.0,0.0,39.0,64.0,1277.0
7,L3,2017-12-31 23:59:00,5080,3,0,474955,0,107600,7,2.0,1.0,133.0,100.0,1712.0
8,L3,2017-12-31 23:59:00,5080,1,0,474956,2,,,NaN,NaN,NaN,NaN,NaN
9,L3,2018-01-01 00:01:00,5080,3,0,1,0,89752,4,2.0,1.0,148.0,67.0,2893.0
